In [ ]:
ML ASSIGNMENT-5
ROLL NUMBER: 187159
NAME: SHANU KUMAR
Q5. Implement a machine learning program to play 5× 5 Tic tac toe game. 
The program should use least means square learning rule.

In [1]:
import numpy as np
import random
import copy

This Board Size parameter can be varied and set to any value

In [2]:
BoardSize = 5

Initializing board to blank characters

In [3]:
def initializeBoard():
    return [[' ' for _ in range(BoardSize)] for _ in range(BoardSize)]

This function checks whether all symbols in a row are the same or not, indicating end of game

In [4]:
def checkRow(board, symbol):
    #Checking for same symbol in a row
    for i in range(BoardSize):
        colFlag = True
        for j in range(BoardSize):
            colFlag = colFlag and (board[i][j]==symbol)
        if(colFlag):
            return True
    return False

This function checks whether all symbols in a column are the same or not, indicating end of game

In [5]:
def checkColumn(board, symbol):
    #Checking for same symbol in a column
    for j in range(BoardSize):
        rowFlag = True
        for i in range(BoardSize):
            rowFlag = rowFlag and (board[i][j]==symbol)
        if(rowFlag):
            return True
    return False

This function checks whether all symbols in either of the diagonals are the same or not, indicating end of game

In [6]:
def checkDiagonals(board, symbol):
    #Checking for same symbol in a diagonal
    diag1Flag,diag2Flag = True, True
    for i in range(BoardSize):
        diag1Flag = diag1Flag and (board[i][i]==symbol)
        diag2Flag = diag2Flag and (board[BoardSize-1-i][i]==symbol)
    return (diag1Flag or diag2Flag)

Checks if game has reached its end

In [7]:
def isGameOver(board, symbol):
    flag = (board==-1) #-1 used to indicate if game as already ended
    flag = flag or checkRow(board,symbol) or checkColumn(board, symbol) or checkDiagonals(board, symbol)
    flag = flag or (' ' not in np.array(board).flatten()) # all entries are filled (drawn state)
    return flag

Returns an array of all possible next moves

In [8]:
def getLegalMoves(boardState, symbol):
    
    legalMoves = []
    for i in range(len(boardState[0])):
        for j in range(len(boardState[0])):
            if(boardState[i][j] == ' '):
                tempBoard = copy.deepcopy(boardState)
                tempBoard[i][j]=symbol
                legalMoves.append(tempBoard)
    return legalMoves

Extracts feature vector. Each feature is defined as:

For ($1\le i \le BoardSize$):

$x_i$ = sum of the counts of number of "symbol1" in all rows or columns is i

For ($BoardSize + 1 \le i \le 2*BoardSize$):

$x_i$ = sum of the counts of number of "symbol2" in all rows or columns is i

In [9]:
def getFeatures(board, symbol1, symbol2):
    
    x = np.zeros(2*BoardSize+1)
    x[0] = 1
    #Extracting row features
    for i in range(BoardSize):
        cntSymbol1,cntSymbol2 = 0,0
        for j in range(BoardSize):
            if(board[i][j]==symbol1):
                cntSymbol1+=1
            elif(board[i][j]==symbol2):
                cntSymbol2+=1
        if(cntSymbol1>=1):
            x[cntSymbol1]+=1
        if(cntSymbol2>=1):
            x[cntSymbol2+BoardSize]+=1
    
    #Extracting column features
    for j in range(BoardSize):
        cntSymbol1,cntSymbol2 = 0,0
        for i in range(BoardSize):
            if(board[i][j]==symbol1):
                cntSymbol1+=1
            elif(board[i][j]==symbol2):
                cntSymbol2+=1
        if(cntSymbol1>=1):
            x[cntSymbol1]+=1
        if(cntSymbol2>=1):
            x[cntSymbol2+BoardSize]+=1
            
    return x

This function prints the board in a visually sound format

In [10]:
def printBoard(board):
    for i in range(BoardSize):
        for j in range(BoardSize):
            print(board[i][j],end='|')
        print()
        for _ in range(2*BoardSize):
            print("-",end='')
        print()

Computes weighted sum of weight vector and feature vector to get the Non-Final Board state

$ z = \sum_{i=0}^{n}w_{i}.x_{i}$

In [11]:
def getNonFinalBoardScore(weightVector,featureVector):
    weightVector = np.array(weightVector).reshape((len(weightVector),1))
    featureVector = np.array(featureVector).reshape((len(featureVector),1))
    boardScore = np.dot(weightVector.T,featureVector)
    return(boardScore[0][0])

Chooses the next move based on the board state which has maximum board score

In [12]:
def chooseMove(board,symbol1,symbol2,weightVector):

    legalMoves = getLegalMoves(board,symbol1)
    legalMoveScores = [getNonFinalBoardScore(weightVector,
                                             getFeatures(i,symbol1,symbol2)) for i in legalMoves]
    return legalMoves[np.argmax(legalMoveScores)]

Random move choice made by the opponent the computer is playing with in the training phase, as the model learns by playing itself

In [13]:
def chooseRandomMove(board,symbol):
    legalMoves = getLegalMoves(board,symbol)
    return random.choice(legalMoves)

The model plays a game with itself where the opponent is a random move chooser. This function generates and stores the game history of a single game from start to finish, used for training.

In [14]:
def getGameHistory(symbols,weightVector1,weightVector2,board):
    gameHistory = []
    gameStatusFlag = True
    
    tempBoard = copy.deepcopy(board)
    while(gameStatusFlag):
        tempBoard = chooseMove(tempBoard,symbols[0],symbols[1],weightVector1)
        gameHistory.append(tempBoard)
        gameStatusFlag = not isGameOver(tempBoard,symbols[0])
        if(gameStatusFlag == False):
            break
        tempBoard = chooseRandomMove(tempBoard,symbols[1])
        gameHistory.append(tempBoard)
        gameStatusFlag =  not isGameOver(tempBoard,symbols[1])
    return gameHistory

Assigning +100 value to a final winning board state and -100 to final losing board state.
For a drawn game, value 0 is assigned.

In [15]:
def getFinalBoardScore(board,symbol1,symbol2):

    # If game ends in a draw
    score = 0
    # If player-1 wins
    if(checkRow(board,symbol1) or checkColumn(board, symbol1) or checkDiagonals(board, symbol1)):
        score = 100
    # If player-2 (i.e opponent) wins
    elif(checkRow(board,symbol2) or checkColumn(board, symbol2) or checkDiagonals(board, symbol2)):
        score = -100
    
    return score

Computes scores of all board states in the game history and returns it as "TrainingData"

In [16]:
def getTrainingSamples(weightVector,symbol1,symbol2,gameHistory):

    trainingData=[]
    for i in range(len(gameHistory)-1):
        featureVector = getFeatures(gameHistory[i+1],symbol1,symbol2)
        trainingData.append([featureVector,getNonFinalBoardScore(weightVector,featureVector)])
    trainingData.append([getFeatures(gameHistory[-1],symbol1,symbol2),
        getFinalBoardScore(gameHistory[-1],symbol1,symbol2)])
    return trainingData

Returns count of wins, losses and drawn games

In [17]:
def getGameStatusCount(symbol1,symbol2,gameStatusCount,gameHistory):

#     for board in gameHistory:
#         printBoard(board)
    finalScore = getFinalBoardScore(gameHistory[-1],symbol1,symbol2)
    if(finalScore == 100):
#         print(symbol1 + " wins")
        gameStatusCount[0] += 1
    elif(finalScore == -100):
#         print(symbol2 + " wins")
        gameStatusCount[1] += 1
    else:
#         print("Draw")
        gameStatusCount[2] += 1
    return gameStatusCount

Least Mean Square (LMS) weight update rule is applied here as:

$W_i = W_i + \alpha * (\hat{V}(boardState)-\hat{V}(Successor(boardState))) * X $

Here $\hat{V}(b)$ is the real valued score assigned to board state b

$\alpha$ is the learning rate

$Successor(b)$ is the next board state

In [18]:
def LMSRule(weightVector,trainData,lRate=0.01):
    for t in trainData:
        vTrainBoardState = t[1]
        vHatBoardState = getNonFinalBoardScore(weightVector,t[0])
        weightVector = weightVector + (lRate * (vTrainBoardState - vHatBoardState) * np.array(t[0]))
    return weightVector

In [19]:
symbols = ('X','O')
numTrainingExamples = 5000

### Training the Model

Using all function defined above:

The *Experiment Generator* gives the initial board state.

The *Performance System* solves the task, here it is playing a game of Tic-Tac-Toe by using learnt weights.

The *Critic* takes input the game history and outputs set of training samples of the target function.

The *Generalizer* here is the LMS algorithm that takes input the generated training samples and updates the weights.

In [20]:
weightVectors = [np.random.rand(2*BoardSize+1),np.random.rand(2*BoardSize+1)]
gameStatusCount = [0,0,0]

for _ in range(numTrainingExamples):
    
    # Experiment Generator
    initialBoardState = initializeBoard()

    # Performance System
    gameHistory = getGameHistory(symbols,weightVectors[0],weightVectors[1],initialBoardState)

    # Critic
    trainDataPlayer1 = getTrainingSamples(weightVectors[0],symbols[0],symbols[1],gameHistory)
    trainDataPlayer2 = getTrainingSamples(weightVectors[1],symbols[1],symbols[0],gameHistory)
    
    # Display board states
    gameStatusCount = getGameStatusCount(symbols[0],symbols[1],gameStatusCount,gameHistory)

    # Generalizer
    weightVectors = [LMSRule(weightVectors[0],trainDataPlayer1),LMSRule(weightVectors[1],trainDataPlayer1)]


print("\nTraining Results: (" + "#Games Player-1 Wins = " + str(gameStatusCount[0]) +
    ", #Games Player-2 Wins = " + str(gameStatusCount[1]) + ", Games Drawn = " + str(gameStatusCount[2]) +
    ")\n")

# Weight Learnt from previous games
learntWeight =  list(np.mean(np.array([weightVectors[0],weightVectors[1]]),axis = 0))
print("Final Learnt Weight Vector: \n"+ str(learntWeight))




Training Results: (#Games Player-1 Wins = 3373, #Games Player-2 Wins = 55, Games Drawn = 1572)

Final Learnt Weight Vector: 
[5.026693194964783, 9.10607210984238, -1.4698048176273162, -6.640879559375607, -10.678978659933117, 53.72508888756845, 0.6671327799896415, -1.7430541404941593, 8.260418442564367, 6.909356276392953, 0.8071862281504834]


### Play the TIC-TAC-TOE game with the learned model

In [25]:
print("\nStart Playing with Computer?(y/n)")
ans = input() 
while(ans == "y"):
    
    boardState = initializeBoard()
    gameStatusFlag = True
    gameHistory = []
    print("Choose symbol: X or O")
    humanSymbol = input()
    if(humanSymbol=='X'):
        computerSymbol = 'O'
    else:
        computerSymbol = 'X'
    print("\nGame begins!!!\n")
    
    while(gameStatusFlag):
        
        if(humanSymbol=='X'):
            print('Your Turn:\n')
            print('Enter Row number (0-',BoardSize-1,")")
            x = int(input())
            print('Enter Column number (0-',BoardSize-1,")")
            y = int(input())
            
            boardState[x][y] = humanSymbol
            printBoard(boardState)
            gameHistory.append(boardState)
            gameStatusFlag =  not isGameOver(boardState,humanSymbol)
            if(gameStatusFlag == False):
                break
            boardState = chooseMove(boardState,computerSymbol,humanSymbol,learntWeight)
            print('Computers\'s Turn:\n')
            printBoard(boardState)
            gameHistory.append(boardState)
            gameStatusFlag = not isGameOver(boardState,computerSymbol)
            
        else:
            boardState = chooseMove(boardState,computerSymbol,humanSymbol,learntWeight)
            print('Computers\'s Turn:\n')
            printBoard(boardState)
            gameHistory.append(boardState)
            gameStatusFlag = not isGameOver(boardState,computerSymbol)
            if(gameStatusFlag == False):
                break
            print('Your Turn:\n')
            print("Enter Row number (0-",BoardSize-1,")")
            x = int(input())
            print('Enter Column number (0-',BoardSize-1,")")
            y = int(input())

            boardState[x][y] = humanSymbol
            printBoard(boardState)
            gameHistory.append(boardState)
            gameStatusFlag =  not isGameOver(boardState,humanSymbol)
    
    if(isGameOver(boardState,computerSymbol)):
        print("Computer Wins!!!")
    elif(isGameOver(boardState,humanSymbol)):
        print("You Win!!!")
    else:
        print("Game is drawn!!!")
    print("Do you want to play another game?(y/n).")
    ans = input()
    if(ans != 'y'):
        break


Start Playing with Computer?(y/n)
y
Choose symbol: X or O
X

Game begins!!!

Your Turn:

Enter Row number (0- 4 )
0
Enter Column number (0- 4 )
0
X| | | | |
----------
 | | | | |
----------
 | | | | |
----------
 | | | | |
----------
 | | | | |
----------
Computers's Turn:

X|O| | | |
----------
 | | | | |
----------
 | | | | |
----------
 | | | | |
----------
 | | | | |
----------
Your Turn:

Enter Row number (0- 4 )
1
Enter Column number (0- 4 )
1
X|O| | | |
----------
 |X| | | |
----------
 | | | | |
----------
 | | | | |
----------
 | | | | |
----------
Computers's Turn:

X|O| | | |
----------
O|X| | | |
----------
 | | | | |
----------
 | | | | |
----------
 | | | | |
----------
Your Turn:

Enter Row number (0- 4 )
2
Enter Column number (0- 4 )
2
X|O| | | |
----------
O|X| | | |
----------
 | |X| | |
----------
 | | | | |
----------
 | | | | |
----------
Computers's Turn:

X|O| | | |
----------
O|X| | | |
----------
 | |X|O| |
----------
 | | | | |
----------
 | | | | |
---------